In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
path = '..'
os.chdir(path)
print(os.getcwd())

In [2]:
astronomy_df = pd.read_csv('mmlu_data/astronomy_questions.csv')
econometrics_df = pd.read_csv('mmlu_data/econometrics_questions.csv')
world_religions_df = pd.read_csv('mmlu_data/world_religions_questions.csv')
complete_questionnaire = pd.concat([astronomy_df, econometrics_df, world_religions_df])
print(complete_questionnaire.shape)

(150, 4)


In [3]:
llama_base_answers = np.load('answers/llama3-8b-base-MMLU-answers.npy')
llama_LR1e4_answers = np.load('answers/llama3-8b-LR1e4-MMLU-answers.npy')
llama_LR5e5_answers = np.load('answers/llama3-8b-LR5e5-MMLU-answers.npy')

In [4]:
complete_questionnaire['base_answers'] = llama_base_answers
complete_questionnaire['LR1e4_answers'] = llama_LR1e4_answers
complete_questionnaire['LR5e5_answers'] = llama_LR5e5_answers

In [5]:
complete_questionnaire.head()

,question,subject,choices,answer,base_answers,LR1e4_answers,LR5e5_answers
0,You've made a scientific theory that there is ...,astronomy,['When you and many other Jedi have tested the...,2,2,2,2
1,One astronomical unit (AU) is equal to approxi...,astronomy,"['130 million km', '150 million km', '170 mill...",1,1,1,1
2,Most rocks on the Moon's surface are older tha...,astronomy,['Lunar rocks are composed of fragments pulver...,1,1,1,```
3,Why can't we see the Sun's corona except durin...,astronomy,"[""The corona is made up mostly of charged part...",3,3\n\n``,,Answer should
4,What type of radiation causes a black hole to ...,astronomy,"['Schwarzschild radiation', 'Planck radiation'...",3,3\n\nAnswer,3,_______________________


In [6]:
complete_questionnaire.to_csv('answers/answered_MMLU_questionnaire.csv', index=False)

In [25]:
complete_questionnaire = pd.read_csv('answers/answered_MMLU_questionnaire_clean.csv')
complete_questionnaire.head()

,question,subject,choices,answer,base_answers,LR1e4_answers,LR5e5_answers
0,You've made a scientific theory that there is ...,astronomy,['When you and many other Jedi have tested the...,2,2,2,2
1,One astronomical unit (AU) is equal to approxi...,astronomy,"['130 million km', '150 million km', '170 mill...",1,1,1,1
2,Most rocks on the Moon's surface are older tha...,astronomy,['Lunar rocks are composed of fragments pulver...,1,1,1,```
3,Why can't we see the Sun's corona except durin...,astronomy,"[""The corona is made up mostly of charged part...",3,3,NaN,NaN
4,What type of radiation causes a black hole to ...,astronomy,"['Schwarzschild radiation', 'Planck radiation'...",3,3,3,_______________________


In [27]:
model_names = ['base','LR1e4','LR5e5']
general_correct_answers = {key: 0 for key in model_names}

for i in range(len(complete_questionnaire)):
    
    expected_answer = str(complete_questionnaire.iloc[i, 3])
    base_answer = complete_questionnaire.iloc[i, 4]
    lr1e4_answer = complete_questionnaire.iloc[i, 5]
    lr5e5_answer = complete_questionnaire.iloc[i, 6]

    if(base_answer == expected_answer):
        general_correct_answers['base'] += 1
    if(lr1e4_answer == expected_answer):
        general_correct_answers['LR1e4'] += 1
    if(lr5e5_answer == expected_answer):
        general_correct_answers['LR5e5'] += 1

general_correct_answers = {key: f'{round(value/len(complete_questionnaire) * 100, 2)}%' for key, value in general_correct_answers.items()}
general_correct_answers

{'base': '62.67%', 'LR1e4': '56.67%', 'LR5e5': '53.33%'}

In [29]:
model_names = ['base','LR1e4','LR5e5']
domains = ['STEM','Humanidades','Ciências Sociais']

domain_specific_correct_answers = {model_name: {domain: 0 for domain in domains} for model_name in model_names}
domain_specific_correct_answers
for i in range(len(complete_questionnaire)):
    if(i < 50):
        domain = 'STEM'
    elif(i >= 50 and i < 100):
        domain = 'Humanidades'
    else:
        domain = 'Ciências Sociais'
    
    expected_answer = str(complete_questionnaire.iloc[i, 3])
    base_answer = complete_questionnaire.iloc[i, 4]
    lr1e4_answer = complete_questionnaire.iloc[i, 5]
    lr5e5_answer = complete_questionnaire.iloc[i, 6]

    if(base_answer == expected_answer):
        domain_specific_correct_answers['base'][domain] += 1
    if(lr1e4_answer == expected_answer):
        domain_specific_correct_answers['LR1e4'][domain] += 1
    if(lr5e5_answer == expected_answer):
        domain_specific_correct_answers['LR5e5'][domain] += 1

domain_specific_correct_answers = {model_name: {domain: f'{round(value/50 * 100, 2)}%' for domain, value in domain_specific_correct_answers[model_name].items()} for model_name in domain_specific_correct_answers.keys()}
domain_specific_correct_answers

{'base': {'STEM': '64.0%',
  'Humanidades': '34.0%',
  'Ciências Sociais': '90.0%'},
 'LR1e4': {'STEM': '54.0%',
  'Humanidades': '48.0%',
  'Ciências Sociais': '68.0%'},
 'LR5e5': {'STEM': '42.0%',
  'Humanidades': '40.0%',
  'Ciências Sociais': '78.0%'}}